In [2]:
#import libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pandas as pd

In [3]:
def getData(url):

    #create connection to url
    html = urllib.request.urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    #read all talbes in list
    tables = soup.find_all("table")
    
    #create empty data array
    data = []

    #read all rows and data of the first table
    for tr in tables[0].find_all('tr'):
        try:
            td = tr.find_all('td')
            label = td[2].text
            
            #only add valid rows
            if "Not assigned" not in label:
                
                if "M" in td[0].text:
                    row = []
                    row.append(td[0].text)
                    row.append(td[1].text)
                    row.append(td[2].text.replace("\n",""))
                    
                    data.append(row)
             

    

        except: 
            pass

    #create data frame
    df = pd.DataFrame(data, columns = ['PostalCode', 'Borough','Neighborhood'])
    
    #group array and combine Neighboorhoods
    df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
    
    return df
        
            

In [4]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#create dataframe from url
df = getData(url)

#print shape of dataframe
print(df.shape)

(102, 3)


In [6]:
dfcoordinates = pd.read_csv("D:\\Backup\\python\\data_files\\Geospatial_Coordinates.csv")

In [8]:
dfcoordinates.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_result = df.merge(dfcoordinates, how="inner", left_on="PostalCode", right_on="Postal Code")

In [10]:
df_result.head(5)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
